# Initializations

In [0]:
!pip install -U --pre tensorflow=="2.*"
!pip install pycocotools
import os, pathlib

from google.colab import drive
path = '/content/drive/My Drive/Colab Notebooks/Object Detection'
drive.mount('/content/drive')

if os.getcwd() != path:
    os.chdir(path)

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

from IPython.display import clear_output
clear_output

Compile protobufs and install the object_detection package ( ~ 10 mins ... )

In [0]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [0]:
%%bash 
cd models/research
pip install .

clear_output

### Imports

In [0]:
import numpy as np
import tensorflow as tf
import os, sys, tarfile, zipfile
import six.moves.urllib as urllib

from PIL import Image
from io import StringIO
from matplotlib import pyplot as plt
from collections import defaultdict
from IPython.display import display
from object_detection.utils import ops as utils_ops, label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

By default I used an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies. Let us write the loader funtion for the specified model :

In [0]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(fname=model_name, origin=base_url + model_file, untar=True)
    model_dir = pathlib.Path(model_dir)/"saved_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']

    return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")) )
# TEST_IMAGE_PATHS.extend( sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.png")) ) )
TEST_IMAGE_PATHS

# Detection

Load an object detection model:

In [0]:
model_name = 'ssd_mobilenet_v1_coco_2018_01_28'
detection_model = load_model(model_name)


Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [0]:
detection_model.inputs
print('Input must be a tensor (!)')

Add a wrapper function to call the model, and cleanup the outputs:

In [0]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)  # input must be Tensor (!)
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.

    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks( 
            output_dict['detection_masks'], output_dict['detection_boxes'], image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

Run it on each test image and show the results:

In [0]:
import cv2, pdb
# pdb.set_trace()

def show_inference(model, image_path):
    image_org = np.array(Image.open(image_path))
    wy, hx = image_org.shape[0:2]
    # print('Image sizes: %d x %d' % (hx, wy) )
    rsz_fct = 0.5           # Image downsize factor 
    img_dsz = ( int(rsz_fct*hx), int(rsz_fct*wy) )         # Image downsize factor 
    image_np = cv2.resize(image_org, dsize=img_dsz, interpolation=cv2.INTER_CUBIC)
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection
    vis_util.visualize_boxes_and_labels_on_image_array(image_np, output_dict['detection_boxes'],
        output_dict['detection_classes'], output_dict['detection_scores'],
        category_index, instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True, line_thickness=7)

    display(Image.fromarray(image_np))  
    cv2.imwrite(os.path.join(path_saved_imgs, 'img_'+str(counter+1)+'.png') , image_np)

In [0]:
path_saved_imgs = path + '/saved_images'
if not os.path.exists(path_saved_imgs):
    os.makedirs(path_saved_imgs)

for counter, image_path in enumerate(TEST_IMAGE_PATHS):
    show_inference(detection_model, image_path, counter)

## Instance Segmentation

In [0]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

masking_model.output_shapes

for image_path in TEST_IMAGE_PATHS:
    show_inference(masking_model, image_path)